## Загрузим нужные библиотеки

In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import numpy as np
from sklearn.metrics import recall_score, precision_score
from catboost import CatBoostClassifier, cv, Pool

from src.seed_ import seed_everything
from src.dicision_stats_features import create_decision

from usr_t_p import create_place_and_score
from src.tfidf_features import create_tfidf_features
from src.inference import calc_result, get_ans

from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

### Fix seed

In [3]:
seed_everything()

In [4]:
df_train = pd.read_csv("data/train_dataset_train.csv")
df_user = pd.read_csv("data/user_decision.csv")

### Добавим новых значений

In [5]:
list_id = df_train["id"].unique().tolist()

df_user_train = df_user[df_user["user_id"].isin(list_id)]
df_user_train

,user_id,period,decision_id
0,10625,1,409
2,10775,4,420
3,10236,2,284
4,10130,3,72
5,10273,2,140
...,...,...,...
60220,10528,1,88
60222,10832,3,315
60224,10189,1,33
60226,10424,2,140


In [6]:
new_datas = create_decision(df_user_train)

In [7]:
train = df_train.copy()

for data in new_datas:
    train = train.merge(data, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)


In [8]:
train.shape

(680, 33)

In [45]:
train

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,cnt_1,cnt_2,cnt_3,cnt_4,max_1,...,median_3,median_4,std_1,std_2,std_3,std_4,sum_1,sum_2,sum_3,sum_4
0,10884,4.0,4.0,4.0,4.0,7.0,14.0,28.0,10.0,386.0,...,125.5,79.5,85.494639,73.199899,100.213232,173.489193,1827.0,1625.0,4141.0,1955.0
1,10106,5.0,4.0,5.0,5.0,21.0,21.0,12.0,8.0,333.0,...,239.0,172.0,83.655786,43.022475,84.584681,160.197869,3691.0,6188.0,2937.0,1378.0
2,10438,6.0,5.0,5.0,5.0,62.0,45.0,63.0,38.0,424.0,...,247.0,321.0,113.514744,115.356671,125.055572,140.129771,12406.0,12350.0,14644.0,10266.0
3,10130,5.0,4.0,5.0,5.0,26.0,28.0,34.0,66.0,410.0,...,193.5,285.0,137.530774,108.084494,122.886065,117.041185,5640.0,3611.0,7089.0,15359.0
4,10667,4.0,5.0,5.0,5.0,58.0,70.0,194.0,35.0,412.0,...,244.5,331.0,115.607017,118.434657,124.114647,73.036287,8526.0,9449.0,44664.0,11437.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,10563,5.0,4.0,5.0,4.0,10.0,2.0,14.0,4.0,422.0,...,328.5,376.5,114.343343,0.000000,133.562205,152.081119,3418.0,552.0,4027.0,1284.0
676,10386,5.0,4.0,5.0,5.0,1.0,14.0,24.0,1.0,114.0,...,157.0,409.0,NaN,96.739482,74.015226,NaN,114.0,2586.0,4318.0,409.0
677,10675,5.0,4.0,5.0,4.0,4.0,17.0,23.0,12.0,434.0,...,304.0,426.5,78.261847,70.534618,80.581410,60.691657,1565.0,6436.0,8004.0,4827.0
678,10366,4.0,4.0,4.0,4.0,10.0,44.0,45.0,38.0,426.0,...,125.0,131.5,68.935074,114.880276,113.471787,104.441480,3246.0,10134.0,7223.0,5990.0


### W2V

In [9]:
# datas = create_w2v_features()

In [10]:
# per1, per2, per3, per4 = datas["period_1"], datas["period_2"], datas["period_3"], datas["period_4"]

In [11]:
# train_w_vec = train.merge(per1, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)
# train_w_vec = train_w_vec.merge(per2, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)
# train_w_vec = train_w_vec.merge(per3, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)
# train_w_vec = train_w_vec.merge(per4, how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)

In [12]:
# train_w_vec

### tfidf

In [13]:
tfidf_features = create_tfidf_features()
# a, b, c, d  = create_tfidf_features()

In [14]:
# tmp = a.drop("period", axis=1).merge(b.drop("period", axis=1), how='inner', left_on="user_id", right_on='user_id')
# tmp = tmp.merge(c.drop("period", axis=1), how='inner', left_on="user_id", right_on='user_id')
# tfidf_features = tmp.merge(d.drop("period", axis=1), how='inner', left_on="user_id", right_on='user_id')
# tfidf_features

In [15]:
train_w_vec = train.merge(tfidf_features.reset_index(), how="left", left_on="id", right_on="user_id").drop("user_id", axis=1)

In [16]:
tfidf_features.to_csv("artifacts/tfidf_features.csv")

### User points

In [19]:
place_score = create_place_and_score()

In [20]:
train_vec_scores = train_w_vec.merge(place_score, how="left", left_on="id", right_on="user_id")

In [21]:
place_score.to_csv("artifacts/place_score.csv", index=False)

### Replace NaN

In [52]:
# train_vec_scores = train_vec_scores.fillna(0)

In [23]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy="mean")
tranformed_data = imp_mean.fit_transform(train_vec_scores)

train_vec_scores_nans = pd.DataFrame(tranformed_data, columns=train_vec_scores.columns)

In [24]:
pd.to_pickle(imp_mean, "artifacts/imp_mean.pkl")

## Выделим выборки

In [26]:
train_vec_scores_nans.to_csv("artifacts/train_vec_scores_nans.csv", index=False)

In [27]:
targets = ["Analytical thinking", "Systemic thinking", "Adaptability", "Focus"]

In [28]:
X = train_vec_scores_nans.drop(targets, axis = 1).drop(["id", "user_id"], axis = 1).drop(592).drop(544)
y = train_vec_scores_nans.drop(592).drop(544)[targets]

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [30]:
y_train_ada = y_train[["Adaptability"]]
y_test_ada = y_test[["Adaptability"]]

y_train_sys = y_train[["Systemic thinking"]]
y_test_sys = y_test[["Systemic thinking"]]

y_train_ana = y_train[["Analytical thinking"]]
y_test_ana = y_test[["Analytical thinking"]]

y_train_foc = y_train[["Focus"]]
y_test_foc = y_test[["Focus"]]

## Обучение модели

In [31]:
params = {"verbose": 250,
          "od_wait": 50,
          "od_type": "Iter",
          "iterations": 2000,
          "depth": 4,
          "learning_rate": 0.007,
          "loss_function": "MultiClass"
          }


model_ada = CatBoostClassifier(**params)
model_sys = CatBoostClassifier(**params)
model_ana = CatBoostClassifier(**params)
model_foc = CatBoostClassifier(**params)

In [32]:
model_ada.fit(X_train, y_train_ada, eval_set=(X_test, y_test_ada))
model_sys.fit(X_train, y_train_sys, eval_set=(X_test, y_test_sys))
model_ana.fit(X_train, y_train_ana, eval_set=(X_test, y_test_ana))
model_foc.fit(X_train, y_train_foc, eval_set=(X_test, y_test_foc))

0:	learn: 1.7820371	test: 1.7826268	best: 1.7826268 (0)	total: 77.2ms	remaining: 2m 34s
250:	learn: 0.9850366	test: 1.0812671	best: 1.0812671 (250)	total: 4.78s	remaining: 33.3s
500:	learn: 0.8463589	test: 1.0028203	best: 1.0028203 (500)	total: 8.82s	remaining: 26.4s
750:	learn: 0.7792510	test: 0.9846834	best: 0.9846225 (748)	total: 13s	remaining: 21.6s
1000:	learn: 0.7246170	test: 0.9777319	best: 0.9777319 (1000)	total: 18.3s	remaining: 18.3s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9772665203
bestIteration = 1058

Shrink model to first 1059 iterations.
0:	learn: 1.5989246	test: 1.5994125	best: 1.5994125 (0)	total: 40ms	remaining: 1m 19s
250:	learn: 0.8380135	test: 0.9213203	best: 0.9213203 (250)	total: 4.07s	remaining: 28.3s
500:	learn: 0.7293261	test: 0.8543990	best: 0.8543990 (500)	total: 8.39s	remaining: 25.1s
750:	learn: 0.6743672	test: 0.8343545	best: 0.8342978 (749)	total: 12.1s	remaining: 20.2s
1000:	learn: 0.6291587	test: 0.8242888	best: 0.8242888 (

## Оценка точности

In [33]:
pred_ada = model_ada.predict(X_test)
pred_sys = model_sys.predict(X_test)
pred_ana = model_ana.predict(X_test)
pred_foc = model_foc.predict(X_test)

pred = pd.DataFrame(index=X_test.index)
pred["Adaptability"] = pred_ada
pred["Systemic thinking"] = pred_sys
pred["Analytical thinking"] = pred_ana
pred["Focus"] = pred_foc

In [35]:
result = 0 

for i, col in enumerate(y_test.columns):
    result += recall_score(y_test[col], pred[col], average='macro', zero_division=True)

print("Recall score", result/4)

Recall score 0.31552117228535


##local##-##public##
0.30658 - 0.261382
0.30897 - 0.258336
0.31063 - 0.271379 - ans6
0.30201 - 0.263627 - ans7
0.31552 - 0.291058 - ans_

In [36]:
model_ada = CatBoostClassifier(**params)
model_sys = CatBoostClassifier(**params)
model_ana = CatBoostClassifier(**params)
model_foc = CatBoostClassifier(**params)

model_ada.fit(X, y["Adaptability"])
model_sys.fit(X, y["Systemic thinking"])
model_ana.fit(X, y["Analytical thinking"])
model_foc.fit(X, y["Focus"])

0:	learn: 1.7820186	total: 50ms	remaining: 1m 39s
250:	learn: 1.0035945	total: 5.94s	remaining: 41.4s
500:	learn: 0.8769046	total: 10.9s	remaining: 32.5s
750:	learn: 0.8182380	total: 15.4s	remaining: 25.5s
1000:	learn: 0.7720311	total: 20.5s	remaining: 20.5s
1250:	learn: 0.7241497	total: 25.3s	remaining: 15.1s
1500:	learn: 0.6792788	total: 30.5s	remaining: 10.1s
1750:	learn: 0.6415471	total: 35.6s	remaining: 5.06s
1999:	learn: 0.6080237	total: 40.6s	remaining: 0us
0:	learn: 1.5979709	total: 23.4ms	remaining: 46.7s
250:	learn: 0.8533128	total: 5.42s	remaining: 37.7s
500:	learn: 0.7486293	total: 10.1s	remaining: 30.2s
750:	learn: 0.6987273	total: 15s	remaining: 24.9s
1000:	learn: 0.6603219	total: 19.3s	remaining: 19.2s
1250:	learn: 0.6220031	total: 24.9s	remaining: 14.9s
1500:	learn: 0.5877514	total: 30.5s	remaining: 10.1s
1750:	learn: 0.5554261	total: 34.9s	remaining: 4.97s
1999:	learn: 0.5278172	total: 39.2s	remaining: 0us
0:	learn: 1.6004096	total: 40.4ms	remaining: 1m 20s
250:	learn:

In [44]:
pd.to_pickle(model_ada, "models/model_ada.pkl")
pd.to_pickle(model_sys, "models/model_sys.pkl")
pd.to_pickle(model_ana, "models/model_ana.pkl")
pd.to_pickle(model_foc, "models/model_foc.pkl")

In [28]:
# FOLDS = 4
#
# ada_pool = Pool(X, y["Adaptability"])
#
# scores_ada = cv(ada_pool,
#                 params,
#                 fold_count=FOLDS, return_models=True)
#
# sys_pool = Pool(X, y["Systemic thinking"])
#
# scores_sys = cv(sys_pool,
#                 params,
#                 fold_count=FOLDS, return_models=True)
#
# ana_pool = Pool(X, y["Analytical thinking"])
#
# scores_ana = cv(ana_pool,
#                 params,
#                 fold_count=FOLDS, return_models=True)
#
# foc_pool = Pool(X, y["Focus"])
#
# scores_foc = cv(foc_pool,
#                 params,
#                 fold_count=FOLDS, return_models=True)

### Inference

In [39]:
test_vec_scores_nans = calc_result()

In [40]:
X = test_vec_scores_nans.drop(targets, axis = 1).drop(["id", "user_id"], axis = 1)
y = test_vec_scores_nans[targets]

y_test_ada = y[["Adaptability"]]

y_test_sys = y[["Systemic thinking"]]

y_test_ana = y[["Analytical thinking"]]

y_test_foc = y[["Focus"]]

In [41]:
pred_ada = model_ada.predict(X)
pred_sys = model_sys.predict(X)
pred_ana = model_ana.predict(X)
pred_foc = model_foc.predict(X)



In [42]:
ans = get_ans(pred_ada, pred_sys, pred_ana, pred_foc)

In [43]:
ans

,id,Analytical thinking,Systemic thinking,Adaptability,Focus
0,10199,4.0,4.0,4.0,4.0
1,10539,4.0,4.0,4.0,4.0
2,10174,4.0,4.0,5.0,4.0
3,10465,4.0,4.0,4.0,4.0
4,10066,4.0,4.0,5.0,4.0
...,...,...,...,...,...
288,10433,4.0,4.0,4.0,4.0
289,10893,4.0,4.0,5.0,5.0
290,10909,4.0,4.0,4.0,4.0
291,10889,4.0,4.0,4.0,4.0
